In [589]:
#Data Preparation
#We will now import some required libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('customer.csv')
dataset.head()


,Customer_ID,Age,Sex,Addtl_HH_size,CustomerType,LastPurchaseAmt,Purchase_Prod_ID,CustomerTier,CustomerTier2,Rewards_Signup
0,315,43.0,male,2,New,26.2500,F.C.C. 13529,2,NaN,0
1,541,36.0,female,2,New,71.0000,WE/P 5735,1,B22,1
2,709,22.0,female,0,New,151.5500,113781,1,NaN,1
3,747,16.0,male,2,New,20.2500,C.A. 2673,3,NaN,0
4,128,24.0,male,0,New,7.1417,C 17369,3,NaN,1


In [647]:
dataset = dataset[['Sex', 'CustomerType', 'CustomerTier', 'Age','Addtl_HH_size', 'Rewards_Signup']]
dataset.head()


,Sex,CustomerType,CustomerTier,Age,Addtl_HH_size,Rewards_Signup
0,male,New,2,43.0,2,0
1,female,New,1,36.0,2,1
2,female,New,1,22.0,0,1
3,male,New,3,16.0,2,0
4,male,New,3,24.0,0,1


In [648]:
#address missing valaue
dataset.fillna(dataset.mean(), inplace=True)
dataset.dropna(inplace=True)

In [629]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,5].values

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder() 
#for 3 columns
for i in range(3):
    X[:,i] = le.fit_transform(X[:,i].astype(str))
    
onehotencoder = OneHotEncoder(categorical_features=[0])
X = onehotencoder.fit_transform(X).toarray()
# Removing the extra dummy variable
X = X[:, 1:]

onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()
# Removing the extra dummy variable
X = X[:, 1:]

onehotencoder = OneHotEncoder(categorical_features=[3])
X = onehotencoder.fit_transform(X).toarray()
# Removing the extra dummy variable
X = X[:, 1:]

print(X.shape)
print(X)


(889, 7)
[[ 1.  0.  1. ...  1. 43.  2.]
 [ 0.  0.  1. ...  0. 36.  2.]
 [ 0.  0.  1. ...  0. 22.  0.]
 ...
 [ 0.  1.  1. ...  1. 24.  2.]
 [ 0.  0.  0. ...  0. 40.  2.]
 [ 1.  0.  1. ...  1. 18.  0.]]


In [642]:
#Splitting the data into Training Set and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

#Normalizing the features
#from sklearn.preprocessing import StandardScaler
#sc_X = StandardScaler()
#X_train = sc_X.fit_transform(X_train)
#X_test = sc_X.transform(X_test)

#Fitting Logistic Regression to Training Set
from sklearn.linear_model import LogisticRegression
classifierObj = LogisticRegression(random_state=0)
classifierObj.fit(X_train, y_train)
#Making predictions on the Test Set
y_pred = classifierObj.predict(X_test)

#Model Accuracy
print(classifierObj.score(X_test,y_test))
#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)



0.7865168539325843
[[140  21]
 [ 36  70]]


In [631]:
#Backward Elimination
import statsmodels.api as sm
X = np.append(arr=np.ones((889,1)).astype(int), values=X, axis=1)

X.shape


(889, 8)

In [632]:
X_sig = X[:,[0,1,2,3,4,5,6,7]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit()
obj_OLS.summary()

X_sig = X[:,[0,1,2,3,5,6,7]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit()
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.395
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     96.05
Date:                Mon, 14 Oct 2019   Prob (F-statistic):           7.64e-93
Time:                        02:10:49   Log-Likelihood:                -396.45
No. Observations:                 889   AIC:                             806.9
Df Residuals:                     882   BIC:                             840.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1982      0.053     22.768      0.000       1.095       1.302
x1            -0.1649      0.040     -4.131      0.000      -0.243      -0.087
x2            -0.3608      0.034    -10.761      0.000      -0.427      -0.295
x3            -0.0716      0.030     -2.417      0.016      -0.130      -0.013
x4            -0.5078      0.028    -18.164      0.000      -0.563      -0.453
x5            -0.0059      0.001     -5.462      0.000      -0.008      -0.004
x6            -0.0290      0.008     -3.475      0.001      -0.045      -0.013
==============================================================================
Omnibus:                       39.905   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.270
Skew:                           0.543   Prob(JB):                     2.44e-10
Kurtosis:                       3.126   Cond. No.                         157.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [644]:
#Splitting the data into Training Set and Test Set
X_sig_train, X_sig_test, y_sig_train, y_sig_test = train_test_split(X_sig, y, test_size=0.3, random_state=0)
mlrObj_sig = LogisticRegression(random_state=0)
mlrObj_sig.fit(X_sig_train, y_sig_train)
y_sig_pred = mlrObj_sig.predict(X_sig_test)


#Model Accuracy
print(mlrObj_sig.score(X_sig_test,y_sig_test))
#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_sig_test, y_sig_pred)
print(cm)

0.7940074906367042
[[142  19]
 [ 36  70]]
